#### – Data Collection 


In [ ]:
## 📌 Setup
!pip install snscrape beautifulsoup4 requests --quiet

import os
import snscrape.modules.twitter as sntwitter
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Create output directory
output_dir = "data/mbego raw data"
os.makedirs(output_dir, exist_ok=True)

#### Twitter Scraping

In [ ]:
# Define search query
query = "(USAID funding OR donor cuts OR USAID health) (Kenya) lang:en since:2023-01-01 until:2024-12-31"

tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i > 1000:  # limit to 1000 for testing
        break
    tweets.append({
        "date": tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
        "username": tweet.user.username,
        "text": tweet.content
    })

# Convert to DataFrame
df_tweets = pd.DataFrame(tweets)

# Save to file
tweets_path = os.path.join(output_dir, "tweets_usaid_kenya.json")
df_tweets.to_json(tweets_path, orient="records", lines=True)
print(f"✅ Saved {len(df_tweets)} tweets to: {tweets_path}")


News Headline Scraping (Basic Demo)

In [ ]:
# Example from The Standard (adjust for real pages)
url = "https://www.standardmedia.co.ke/topic/usaid"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# Collect headlines
articles = soup.find_all("h2")  # You may need to update this tag
news_data = [{"headline": a.text.strip()} for a in articles if a.text.strip()]

# Save to JSON
df_news = pd.DataFrame(news_data)
news_path = os.path.join(output_dir, "news_usaid_kenya.json")
df_news.to_json(news_path, orient="records", lines=True)
print(f"✅ Saved {len(df_news)} news headlines to: {news_path}")
